In [4]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

In [5]:
caltech_dir = "./dataset"
categories = ["corona_ex", "prevention_ex", "do_ex", "fighting_ex"]
nb_classes = len(categories)

In [6]:
image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

In [7]:
for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식

corona_ex  파일 길이 :  293
corona_ex  :  ./dataset/corona_ex\new_0_1026.jpg
prevention_ex  파일 길이 :  296
prevention_ex  :  ./dataset/prevention_ex\new_0_1082.jpg
do_ex  파일 길이 :  293
do_ex  :  ./dataset/do_ex\new_0_100.jpg
fighting_ex  파일 길이 :  290
fighting_ex  :  ./dataset/fighting_ex\new_0_1047.jpg


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./dataset.npy", xy )

print("ok", len(y))

ok 1172


C:\Users\soski\anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [12]:
import tensorflow as tf
from tensorflow import keras
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras import backend as K

AttributeError: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

In [ ]:

config = tf.compat.v1.ConfigProto() 
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

X_train, X_test, y_train, y_test = np.load('./dataset.npy', allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

In [ ]:
categories = ["corona_ex", "prevention_ex", "do_ex", "fighting_ex"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [ ]:
with K.tf_ops.device('/:GPU:0:'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_dir = './dataset/model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    model_path = model_dir + '/plzsuccess.model'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

In [ ]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_oss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()

In [ ]:
from keras.models import load_model
model.save('conkeras_model.h5')